## Data Pre-processing

In this notebook, we will filter all datasets based on index date.

In [1]:
import pandas as pd
import random

Get diagnosis date from Admissions Table

In [2]:
diagnosis = pd.read_csv('rawdata/DIAGNOSES_ICD.csv')
diagnosis = diagnosis.dropna(subset=['ICD9_CODE'])
admissions = pd.read_csv('rawdata/ADMISSIONS.csv')
diag_merged_time = diagnosis.merge(admissions[['HADM_ID', 'ADMITTIME']])
diag_merged_time['ADMITTIME'] = pd.to_datetime(diag_merged_time['ADMITTIME'])

### Obtain index dates for case and control patients.
We define case patients as patients with an ICD9 code that starts with '428'

Index date for case patient is the first date when they are diagnosed with a '428.x' ICD9 code.
Index date for control patient is the last date of diagnosis record.

In [3]:
# Set of case patient ID
case_set = set(diagnosis[diagnosis['ICD9_CODE'].str.startswith('428').astype(bool)]['SUBJECT_ID'])

In [4]:
# Case patients' index date
diag_time_case = diag_merged_time[diag_merged_time['ICD9_CODE'].str.startswith('428')]
case_index_date = diag_time_case[['SUBJECT_ID', 'ADMITTIME']].groupby('SUBJECT_ID', as_index=False).agg({'ADMITTIME': 'min'})
case_index_date.rename(columns={'ADMITTIME':'INDEXDATE'}, inplace=True)

In [5]:
# Control patients' index date
diag_time_control = diag_merged_time[~diag_merged_time['SUBJECT_ID'].isin(case_set)]
control_index_date = diag_time_control[['SUBJECT_ID', 'ADMITTIME']].groupby('SUBJECT_ID', as_index=False).agg({'ADMITTIME': 'max'})
control_index_date.rename(columns={'ADMITTIME':'INDEXDATE'}, inplace=True)

In [6]:
# Full data for index dates
all_index_date = pd.concat([case_index_date, control_index_date], ignore_index=True)

### Note Events data

In [7]:
noteevents = pd.read_csv('rawdata/NOTEEVENTS.csv')

C:\Users\Keith\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
# Split notes into case and control datasets
noteevents_case = noteevents[noteevents['SUBJECT_ID'].isin(case_set)]
noteevents_control = noteevents[~noteevents['SUBJECT_ID'].isin(case_set)]

In [9]:
# Merge notes with respective index dates
note_case_indexdate = noteevents_case.merge(case_index_date, how='left')
note_control_indexdate = noteevents_control.merge(control_index_date, how='left')
note_control_indexdate = note_control_indexdate[~note_control_indexdate.INDEXDATE.isnull()] # Remove notes without index date

In [39]:
# Filter notes, remove notes outside observation window
note_case_indexdate['CHARTDATE'] = pd.to_datetime(note_case_indexdate['CHARTDATE'])
note_control_indexdate['CHARTDATE'] = pd.to_datetime(note_control_indexdate['CHARTDATE'])

filter_note_case = note_case_indexdate[note_case_indexdate.CHARTDATE < note_case_indexdate.INDEXDATE - pd.Timedelta(days=10)]
filter_note_control = note_control_indexdate[note_control_indexdate.CHARTDATE < note_control_indexdate.INDEXDATE - pd.Timedelta(days=10)]

In [11]:
print("Number of patients in case: ", len(set(filter_note_case.SUBJECT_ID)))
print("Number of notes in case: ", len(filter_note_case))
print("Number of patients in control: ", len(set(filter_note_control.SUBJECT_ID)))
print("Number of notes in control: ", len(filter_note_control))

Number of patients in case:  2909
Number of notes in case:  70927
Number of patients in control:  7832
Number of notes in control:  226664


We see that the ratio for case-control patients is slightly skewed, thus we will randomly drop some control patients in order to achieve a 40-60 ratio for case-control patients

In [12]:
# We will drop control patients in order to get 4364 control patients for our final dataset
num_control_patients = round(2909 / 4 * 6)

In [13]:
# Set of patient_id for case and control events
noteevents_case_id_set = set(filter_note_case['SUBJECT_ID'])
noteevents_control_id_set = set(filter_note_control['SUBJECT_ID'])

In [14]:
# Get 4364 control patient_id
random.seed(1)
control_id_subset = random.sample(noteevents_control_id_set, num_control_patients)

In [40]:
# Cut down number of control patients and notes
filter_note_control_subset = filter_note_control[filter_note_control.SUBJECT_ID.isin(control_id_subset)]

In [16]:
print("Number of patients in case: ", len(set(filter_note_case.SUBJECT_ID)))
print("Number of notes in case: ", len(filter_note_case))
print("Number of patients in control: ", len(set(filter_note_control_subset.SUBJECT_ID)))
print("Number of notes in control: ", len(filter_note_control_subset))

Number of patients in case:  2909
Number of notes in case:  70927
Number of patients in control:  4364
Number of notes in control:  125499


### Patient IDs

Since our main data of interest is text data, we will get the list of case and control patient IDs off noteevents table. We will also filter the other datasets based on this list of patient IDs in Spark

In [17]:
case_id = noteevents_case_id_set # 2909 case
control_id = set(control_id_subset) # 4364 control
all_id = case_id|control_id # total 7273 patients

### Filter datasets based on index dates

We define our observation window as all data up till 10 days before index date.

We will filter out data outside our observation window for prescriptions, lab, procedures, and noteevents dataset.

In [18]:
# Load all relevant datasets to filter
prescriptions = pd.read_csv('rawdata/PRESCRIPTIONS.csv')
lab = pd.read_csv('rawdata/LABEVENTS.csv')
procedures = pd.read_csv('rawdata/CPTEVENTS.csv')

C:\Users\Keith\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Keith\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4,5,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
# Filter diagnosis
diag_indx = diag_merged_time.merge(all_index_date)
filter_diag = diag_indx[diag_indx.ADMITTIME < (diag_indx.INDEXDATE - pd.Timedelta(days=10))]

In [23]:
# Filter prescriptions
prescriptions['STARTDATE'] = pd.to_datetime(prescriptions['STARTDATE'])
prescriptions_indx = prescriptions.merge(all_index_date)
filter_presc = prescriptions_indx[prescriptions_indx.STARTDATE < (prescriptions_indx.INDEXDATE - pd.Timedelta(days=10))]

In [24]:
# Filter lab
lab['CHARTTIME'] = pd.to_datetime(lab['CHARTTIME'])
lab_indx = lab.merge(all_index_date)
filter_lab = lab_indx[lab_indx.CHARTTIME < (lab_indx.INDEXDATE - pd.Timedelta(days=10))]

In [25]:
# Filter procedures
proc_merged_time = procedures.merge(admissions[['HADM_ID', 'ADMITTIME']])
proc_merged_time['ADMITTIME'] = pd.to_datetime(diag_merged_time['ADMITTIME'])
proc_indx = proc_merged_time.merge(all_index_date)
filter_proc = proc_indx[proc_indx.ADMITTIME < (proc_indx.INDEXDATE - pd.Timedelta(days=10))]

In [ ]:
# Export notes
filter_note_case.to_csv('3_code/data/case_notes.csv', index=False)
filter_note_control_subset.to_csv('3_code/data/control_notes.csv', index=False)

In [29]:
# Export diagnosis, prescriptions, lab, and procedures in observation window
filter_diag.to_csv('3_code/data/filter_diag.csv', index=False)
filter_presc.to_csv('3_code/data/filter_presc.csv', index=False)
filter_lab.to_csv('3_code/data/filter_lab.csv', index=False)
filter_proc.to_csv('3_code/data/filter_proc.csv', index=False)

In [41]:
del filter_note_case['TEXT']
del filter_note_control_subset['TEXT']

# Export notes without text
filter_note_case.to_csv('3_code/data/case_notes_no_text.csv', index=False)
filter_note_control_subset.to_csv('3_code/data/control_notes_no_text.csv', index=False)